In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [3]:
print(os.listdir("/gdrive/My Drive/Banana_Ripeness_Classification"))

['unripe', 'ripe', 'rotten', 'overripe']


In [4]:
DATADIR = '/gdrive/My Drive/Banana_Ripeness_Classification'
CATEGORIES =["overripe","ripe","rotten","unripe"]
IMG_SIZE = 100

In [5]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                print("hata")
create_all_data()

In [6]:
print("toplam görüntü sayısı: ",len(all_data))

toplam görüntü sayısı:  13478


In [7]:
X = []
y = []
for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE * IMG_SIZE * 3) # Düzleştirme işlemi
y = np.array(y)

In [8]:
# Veriyi bölme
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [10]:
# Etiketleri one-hot kodlama
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)

In [9]:
from keras.models import Sequential
from keras.layers import Dense

In [11]:
# Modeli oluşturma
model = Sequential()
model.add(Dense(1024, input_shape=(IMG_SIZE * IMG_SIZE * 3,), activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(4, activation='softmax')) # Çıkış katmanı 4 nöronlu olmalı ve 'softmax' kullanmalı


In [12]:
# Modeli derleme
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [13]:
# Modeli eğitme
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
266/266 [==============================] - 8s 17ms/step - loss: 468.4783 - accuracy: 0.4397 - val_loss: 8.0329 - val_accuracy: 0.6684
Epoch 2/10
266/266 [==============================] - 3s 13ms/step - loss: 12.5708 - accuracy: 0.5799 - val_loss: 13.2103 - val_accuracy: 0.4333
Epoch 3/10
266/266 [==============================] - 4s 13ms/step - loss: 7.3528 - accuracy: 0.5781 - val_loss: 3.8067 - val_accuracy: 0.6822
Epoch 4/10
266/266 [==============================] - 4s 14ms/step - loss: 3.9275 - accuracy: 0.6217 - val_loss: 2.4594 - val_accuracy: 0.6758
Epoch 5/10
266/266 [==============================] - 3s 13ms/step - loss: 2.2531 - accuracy: 0.6547 - val_loss: 3.9932 - val_accuracy: 0.5975
Epoch 6/10
266/266 [==============================] - 3s 13ms/step - loss: 8.8731 - accuracy: 0.5382 - val_loss: 4.8216 - val_accuracy: 0.5307
Epoch 7/10
266/266 [==============================] - 3s 13ms/step - loss: 2.6823 - accuracy: 0.6170 - val_loss: 1.5755 - val_accuracy: 0.

In [14]:
# Test seti üzerinde değerlendirme
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test, axis=1)

127/127 [==============================] - 0s 2ms/step


In [17]:
# Performans metrikleri
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted", zero_division=0)
recall = recall_score(y_true, y_pred, average="weighted", zero_division=0)
f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)


In [18]:
# Sonuçları gösterme
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))

Accuracy:  0.3365479723046489
Precision:  0.39467035201444134
Recall:  0.3365479723046489
F1 Score:  0.22205808930669213

Confusion Matrix:
 [[ 106  687   29    0]
 [  28 1165    0    0]
 [ 314  954   90    0]
 [   1  667    3    0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.24      0.13      0.17       822
           1       0.34      0.98      0.50      1193
           2       0.74      0.07      0.12      1358
           3       0.00      0.00      0.00       671

    accuracy                           0.34      4044
   macro avg       0.33      0.29      0.20      4044
weighted avg       0.39      0.34      0.22      4044



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
